In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tarfile
import cv2
import pickle
%matplotlib inline

In [2]:
path = 'C:/Users/proti/256_ObjectCategories/'

In [3]:
os.chdir(path)

In [4]:
folders = os.listdir()

In [5]:
print(len(folders))

257


In [6]:
folder_paths = []
all_images = []
all_classes = []

In [7]:
len(all_images)

0

In [8]:
img_size = 128

In [9]:
from PIL import Image

def make_square(image, min_size=img_size, fill_color=(0, 0, 0, 0)):
    size = (min_size, min_size)
    image.thumbnail(size, Image.ANTIALIAS)
    background = Image.new('RGB', size, (255, 255, 255, 0))
    background.paste(
        image, (int((size[0] - image.size[0]) / 2), int((size[1] - image.size[1]) / 2))
    )

    new_img = np.array(background)
    new_img.flatten()
    return new_img

In [10]:
for folder in range(10):
    folder_paths = path+str(folders[folder])+str('/')
    
    os.chdir(folder_paths)
    image_in_folder = os.listdir()

    for image in range(len(image_in_folder)):
        img = Image.open(image_in_folder[image])
        img = make_square(img)
        
        all_images.append(img.flatten()/255)
        all_classes.append(folders[folder])

In [11]:
len(all_images)

1245

In [12]:
len(all_classes)

1245

In [13]:
print(all_classes[len(all_classes)-1])

010.beer-mug


In [14]:
all_images[0].shape

(49152,)

In [15]:
len(all_images)

1245

In [16]:
# changing all images to dataframe where each row is an image and columns are the pixel values
all_images_df = np.asarray(all_images)

In [17]:
all_images_df.shape

(1245, 49152)

In [18]:
set(all_classes)

{'001.ak47',
 '002.american-flag',
 '003.backpack',
 '004.baseball-bat',
 '005.baseball-glove',
 '006.basketball-hoop',
 '007.bat',
 '008.bathtub',
 '009.bear',
 '010.beer-mug'}

In [22]:
all_classes.head()

,001.ak47,002.american-flag,003.backpack,004.baseball-bat,005.baseball-glove,006.basketball-hoop,007.bat,008.bathtub,009.bear,010.beer-mug
0,1,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0


In [19]:
all_classes = pd.get_dummies(all_classes) 

In [20]:
set(all_classes)

{'001.ak47',
 '002.american-flag',
 '003.backpack',
 '004.baseball-bat',
 '005.baseball-glove',
 '006.basketball-hoop',
 '007.bat',
 '008.bathtub',
 '009.bear',
 '010.beer-mug'}

In [21]:
all_classes.head()

,001.ak47,002.american-flag,003.backpack,004.baseball-bat,005.baseball-glove,006.basketball-hoop,007.bat,008.bathtub,009.bear,010.beer-mug
0,1,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0


In [23]:
all_images = np.array(all_images)

In [24]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(all_images, all_classes, test_size=0.1)

In [25]:
len(X_test)

125

In [26]:
len(X_train)

1120

In [27]:
X_train = X_train.reshape(-1,img_size,img_size,3)

In [28]:
X_test = X_test.reshape(-1,img_size,img_size,3)

In [29]:
X_train.shape

(1120, 128, 128, 3)

In [30]:
X_test.shape

(125, 128, 128, 3)

In [31]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

Using TensorFlow backend.
C:\Users\proti\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\proti\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\proti\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\proti\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [32]:
generator = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

In [33]:
generator.flow(X_train)

In [34]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
import keras.backend as K
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization
from keras.layers import Conv2D, MaxPool2D, AvgPool2D,Flatten

In [40]:
layer = 3
filterz = 32
lr = .001
epochs = 80
batch_size = 32
decay = lr/epochs
adam = Adam(lr=lr,decay=decay)

model = Sequential()
model.add(Conv2D(filters = (filterz), kernel_size= (3,3), input_shape = (img_size,img_size,3), padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size = (2,2)))
print('filter: '+ str(filterz))

for i in range(layer):
    filterz1 = filterz * (2**(i+1))
    model.add(Conv2D(filters = filterz1, kernel_size= (3,3), padding = 'same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPool2D(pool_size = (2,2)))
    print('filter: '+ str(filterz * (2**(i+1))))

print('\n')
model.add(Flatten())

model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu')) 

model.add(Dense(10,activation='softmax'))
model.compile(adam, 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

filter: 32
filter: 64
filter: 128
filter: 256




In [41]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 128, 128, 32)      896       
_________________________________________________________________
batch_normalization_6 (Batch (None, 128, 128, 32)      128       
_________________________________________________________________
activation_6 (Activation)    (None, 128, 128, 32)      0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 64, 64, 64)        18496     
_________________________________________________________________
batch_normalization_7 (Batch (None, 64, 64, 64)        256       
_________________________________________________________________
activation_7 (Activation)    (None, 64, 64, 64)       

In [42]:
model.fit_generator(generator.flow(X_train, y_train.values, batch_size=batch_size),
                    steps_per_epoch=len(X_train) / batch_size, 
                    epochs=80,
                    verbose=1,
                    validation_data=(X_test, y_test.values))

Epoch 1/80
35/35 [==============================] - 72s 2s/step - loss: 2.4048 - accuracy: 0.3161 - val_loss: 2.6540 - val_accuracy: 0.1120
Epoch 2/80
35/35 [==============================] - 76s 2s/step - loss: 1.6995 - accuracy: 0.4286 - val_loss: 2.9603 - val_accuracy: 0.2000
Epoch 3/80
35/35 [==============================] - 74s 2s/step - loss: 1.6023 - accuracy: 0.4696 - val_loss: 3.0464 - val_accuracy: 0.2480
Epoch 4/80
35/35 [==============================] - 73s 2s/step - loss: 1.5154 - accuracy: 0.5009 - val_loss: 3.7465 - val_accuracy: 0.2160
Epoch 5/80
35/35 [==============================] - 75s 2s/step - loss: 1.4364 - accuracy: 0.5045 - val_loss: 4.8757 - val_accuracy: 0.2160
Epoch 6/80
35/35 [==============================] - 79s 2s/step - loss: 1.4310 - accuracy: 0.5143 - val_loss: 3.9949 - val_accuracy: 0.2160
Epoch 7/80
35/35 [==============================] - 73s 2s/step - loss: 1.3843 - accuracy: 0.5348 - val_loss: 3.0932 - val_accuracy: 0.2560
Epoch 8/80
35/35 [==

35/35 [==============================] - 88s 3s/step - loss: 0.3994 - accuracy: 0.8634 - val_loss: 2.0603 - val_accuracy: 0.5120
Epoch 60/80
35/35 [==============================] - 79s 2s/step - loss: 0.4353 - accuracy: 0.8429 - val_loss: 1.7537 - val_accuracy: 0.5920
Epoch 61/80
35/35 [==============================] - 72s 2s/step - loss: 0.4029 - accuracy: 0.8679 - val_loss: 2.0885 - val_accuracy: 0.5200
Epoch 62/80
35/35 [==============================] - 72s 2s/step - loss: 0.4656 - accuracy: 0.8482 - val_loss: 2.1094 - val_accuracy: 0.5280
Epoch 63/80
35/35 [==============================] - 72s 2s/step - loss: 0.3756 - accuracy: 0.8696 - val_loss: 1.5247 - val_accuracy: 0.6240
Epoch 64/80
35/35 [==============================] - 80s 2s/step - loss: 0.3669 - accuracy: 0.8795 - val_loss: 2.5146 - val_accuracy: 0.4800
Epoch 65/80
35/35 [==============================] - 73s 2s/step - loss: 0.3481 - accuracy: 0.8830 - val_loss: 1.7550 - val_accuracy: 0.5520
Epoch 66/80
35/35 [======